In [3]:
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_google_genai import ChatGoogleGenerativeAI
from pprint import pprint

In [4]:
search_tool = DuckDuckGoSearchRun()
result = search_tool.invoke("what is the whether today?")

In [ ]:
pprint(result)

('localweatheralerts.net has been visited by 100K+ users in the past month '
 'View radar maps, tornado tracking, and more with one-click access. Get '
 'Doppler radar, storm alerts, and NOAA updates in real time—free tool. '
 'Real-Time Updates · Auto- Apply Discounts · Free Coupon Extension · Free '
 'Coupons Types : Hourly, Daily, Weekly, 10-Day Indore, Madhya Pradesh, India '
 'Weather Forecast, with current conditions, wind, air quality, and what to '
 'expect for the next 3 days. 1 day ago · Get the latest hourly weather '
 'updates for Indore today . Detailed forecast including temperature, wind, '
 "rain, snow, and UV index. Stay informed about today ' s weather conditions "
 "in Indore. 3 days ago · Latest weather forecast for Indore for today ' s , "
 "hourly weather forecast, including today 's temperatures in Indore, wind, "
 "rain and more. Yesterday's weather Partly sunny. 83 / 71 °F Humidity: 77%. "
 "Wind: 7 mph ↑ from West More weather last week Nov 19, 2025 · How' s t

In [5]:
llm = ChatGoogleGenerativeAI(model= "models/gemini-2.0-flash")
# llm.invoke("what is the whether today")

## Prompt Hub

In [8]:
from langsmith import Client

In [20]:
client = Client()
prompt = client.pull_prompt("mitchell-compoze/summarize_for_retreival")
# pprint(client.list_prompts())

In [25]:
pprint(dict(prompt))

{'input_types': {'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[langchain_core.messages.function.FunctionMessageChunk, Tag(

## Agent

In [6]:
from langchain.agents import create_agent

In [7]:
agent = create_agent(
    model = llm,
    tools= [search_tool],
    system_prompt= """
    Question: the input question you must answer
    Thought: you should always think about what to do
    Action: the action to take, should be one of [{tool_names}]
    Action Input: the input to the action
    Observation: the result of the action
    ... (this Thought/Action/Action Input/Observation can repeat N times)
    Thought: I now know the final answer
    Final Answer: the final answer to the original input question
    """
)

result = agent.invoke({
    "messages": [{"role": "user", "content": "What is the population of the capital of France? and What is the population of the capital of india and What is the population of the capital of Madhya Pradhesh"}]
})


In [8]:
result

{'messages': [HumanMessage(content='What is the population of the capital of France? and What is the population of the capital of india and What is the population of the capital of Madhya Pradhesh', additional_kwargs={}, response_metadata={}, id='0fbf80c5-fde6-4a6f-b9c5-564a7e5ae7ab'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'duckduckgo_search', 'arguments': '{"query": "population of Bhopal"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--05938f9c-d609-49e4-b362-09652daa1070-0', tool_calls=[{'name': 'duckduckgo_search', 'args': {'query': 'population of Paris'}, 'id': 'fb5d0f12-0c66-4b0a-8b60-86c5c738da32', 'type': 'tool_call'}, {'name': 'duckduckgo_search', 'args': {'query': 'population of New Delhi'}, 'id': '80656797-acf8-4956-944e-27f9407187e9', 'type': 'tool_call'}, {'name': 'duckduckgo_sear

In [32]:
for i in range(len(result['messages'])):
    # print('==================================================================================================== \n ')
    pprint(dict(result['messages'][i]))
    print(' \n ==================================================================================================== \n ')

{'additional_kwargs': {},
 'content': 'What is the population of the capital of France? and What is the '
            'population of the capital of india and What is the population of '
            'the capital of Madhya Pradhesh',
 'id': 'ae8179d3-2bfa-4005-8622-812c72ba8ffd',
 'name': None,
 'response_metadata': {},
 'type': 'human'}
 
 
{'additional_kwargs': {'function_call': {'arguments': '{"query": "population '
                                                      'of Bhopal"}',
                                         'name': 'duckduckgo_search'}},
 'content': '',
 'id': 'lc_run--c714e702-4285-42c4-8797-db795d38418c-0',
 'invalid_tool_calls': [],
 'name': None,
 'response_metadata': {'finish_reason': 'STOP',
                       'model_name': 'gemini-2.0-flash',
                       'model_provider': 'google_genai',
                       'prompt_feedback': {'block_reason': 0,
                                           'safety_ratings': []},
                       'safety_ra

In [40]:
for chunk in agent.stream({
    "messages": [{"role": "user", "content": "What is the population of the capital of France? and What is the population of the capital of india and What is the population of the capital of Madhya Pradhesh"}]
}, stream_mode="values"):
    messages = chunk.get("messages", [])
    if messages:
        last_msg = messages[-1]
        print("===========================LAST MESSAGE================================= \n\n")
        print(last_msg)
        print("============================================================ \n\n")
        # Show reasoning/planning
        if hasattr(last_msg, 'content') and last_msg.content:
            pprint(f"Thought: {last_msg.content}")
        
        # Show tool calls (actions)
        if hasattr(last_msg, 'tool_calls') and last_msg.tool_calls:
            for tool_call in last_msg.tool_calls:
                pprint(f"Action: {tool_call['name']}")
                pprint(f"Action Input: {tool_call['args']}")
                # pprint(f"Action Output: {tool_call['content']}")
        
        # Show tool results (observations)
        if last_msg.type == "tool":
            pprint(f"Observation: {last_msg.content}")


===========================LAST MESSAGE================================= 


content='What is the population of the capital of France? and What is the population of the capital of india and What is the population of the capital of Madhya Pradhesh' additional_kwargs={} response_metadata={} id='304916aa-f518-4480-b8c0-f8a4d236a8aa'


('Thought: What is the population of the capital of France? and What is the '
 'population of the capital of india and What is the population of the capital '
 'of Madhya Pradhesh')
===========================LAST MESSAGE================================= 


content='' additional_kwargs={'function_call': {'name': 'duckduckgo_search', 'arguments': '{"query": "population of Bhopal"}'}} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': [], 'model_provider': 'google_genai'} id='lc_run--8edfd150-e0c0-48d7-abf2-65d5586665d7-0' tool_calls=[{'name': 'duckduckgo_

In [22]:
agents_streamed_output = agent.stream({
    "messages": [{"role": "user", "content": "What is the population of the capital of France? and What is the population of the capital of india and What is the population of the capital of Madhya Pradhesh"}]
}, stream_mode="values")

### Thought -> Action -> Oberservation

In [23]:
messages = None
human = True
for chunk in agents_streamed_output:
    messages = chunk.get("messages", [])
    # print(chunk.get("messages", []))
    if messages:
        last_msg = messages[-1]
        # print(last_msg.content)
        # Reasoning/ Planning
        if hasattr(last_msg, "content") and last_msg.content:
            if human:
                print(f"Human Message: {last_msg.content}")
                human = False
            else:
                print(f"Thought: {last_msg.content}")
            
        # Actions
        if hasattr(last_msg, "tool_calls") and last_msg.tool_calls:
            for tool_call in last_msg.tool_calls:
                print(f"Action: {tool_call['name']},  Search Query: {tool_call['args']}")
        
        # Obervations(Results[tool calling results])
        if last_msg.type == "tool":
            print(f"Observation: {last_msg.content}")
            

Human Message: What is the population of the capital of France? and What is the population of the capital of india and What is the population of the capital of Madhya Pradhesh
Action: duckduckgo_search,  Search Query: {'query': 'population of Paris'}
Action: duckduckgo_search,  Search Query: {'query': 'population of New Delhi'}
Action: duckduckgo_search,  Search Query: {'query': 'population of Bhopal'}
Thought: September 26, 2025 - The district has a population density of 855 inhabitants per square kilometre (2,210/sq mi). Its population growth rate over the decade 2001-2011 was 28.46%. Bhopal has a sex ratio of 918 females for every 1000 males, and a literacy rate of 80.37%. 80.85% of the population lives in urban areas. 5 days ago - Bhopal has been ranked 6th best ... census, the population of the Bhopal city (the area under Bhopal Municipal Corporation) is 1,798,218, with 936,168 males and 862,050 females .... 3 weeks ago - This is a list of urban agglomerations and cities (those no

In [19]:
my_msg = messages

In [20]:
my_msg

[HumanMessage(content='What is the population of the capital of France? and What is the population of the capital of india and What is the population of the capital of Madhya Pradhesh', additional_kwargs={}, response_metadata={}, id='184f0521-90ee-4d8b-bee1-085d8535bed7'),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'duckduckgo_search', 'arguments': '{"query": "population of Bhopal"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--c830b506-2260-47b4-aee9-2b8c078d75ff-0', tool_calls=[{'name': 'duckduckgo_search', 'args': {'query': 'population of Paris'}, 'id': 'd31d0f72-cb92-4bc6-ba53-0793d454e9bd', 'type': 'tool_call'}, {'name': 'duckduckgo_search', 'args': {'query': 'population of New Delhi'}, 'id': '7ca6b7f1-35d1-461e-a76c-61faed0d6d40', 'type': 'tool_call'}, {'name': 'duckduckgo_search', 'args': {

In [98]:
pprint(dict(my_msg['messages'][2]))
pprint(my_msg['messages'])

{'additional_kwargs': {},
 'artifact': None,
 'content': 'The population of the city of Paris reached a historic high of '
            '2.9 million in 1921 but then declined; between 1954 and 1999 it '
            'declined at every ... Learn about the current and projected '
            'population of Paris , the capital of France, based on official '
            'statistics and demographic trends. Find out the gender, age, and '
            'life expectancy structure, as well as the population change and '
            'decline factors. p: Provisional value r: Revised value Scope: 75 '
            '- Paris Source: See documentation view Find out the latest data '
            'on population , age structure, unemployment, real estate and more '
            'in Paris and its region. Compare the cost of living, employment '
            'prospects and cultural offer with the rest of France. Paris , '
            'city and capital of France, located along the Seine River, in the '
         

In [54]:
for msg in messages:
    pprint(msg.content)

('What is the population of the capital of France? and What is the population '
 'of the capital of india and What is the population of the capital of Madhya '
 'Pradhesh')
''
('1 day ago - Paris is the capital and largest city of France, with an '
 'estimated city population of 2,048,472 in an area of 105.4 km2 (40.7 sq mi), '
 'and a metropolitan population of 13,171,056 as of January 2025. Located on '
 'the river Seine in the centre of the Île-de-France region, it is the largest '
 'metropolitan ... 1 day ago - It has a population of 12,213,447 as of January '
 '2018, or 18.3 percent of the population of France. The metropolitan or '
 "functional area (aire d'attraction) of Paris covers 18,941 km2 (7,313 sq mi) "
 'and has 13,064,617 inhabitants (2018). May 12, 2025 - The area is 105 square '
 'kilometres (41 square miles), and around 2.15 million people live there. If '
 'suburbs are counted, the population of the Paris area rises to 10.7 million '
 'people. August 6, 2025 - The a